<a href="https://colab.research.google.com/github/fiony/web_scrape/blob/master/DomainApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Suburb

In [ ]:
TURRAMURRA = Suburb(name="Turramurra", postcode=2074, state='nsw')
LANE_COVE = Suburb(name="Lane Cove", postcode=2066, state='nsw')
ARTARMON = Suburb(name="artarmon", postcode=2064, state='nsw')

In [ ]:
search_price = PriceRange(1.8e6, 3e6)
search_property = PropertyRange(bed="3+", bath="1+", car="1+")

### Property Listing

In [ ]:
sub = ARTARMON#LANE_COVE

In [ ]:
init_logging()
setLogLevel(logging.INFO)

In [ ]:
client = DomainClient(SiteInfo)

In [ ]:
sub_by_property = client.listing(sub, property_range=search_property)

In [ ]:
sub_by_price = client.listing(sub, price_range=search_price)

In [ ]:
Q_GAS = "gas=='Y'"
Q_NO_POOL = "pool=='N'"
Q_4_bed = "bed==4"
Q_SUB = "sub=='%s'"%str.upper(sub.name)
def display_listing(result, query=[]):
  df = to_dataframe(result)
  for q in query:
    df = df.query(q)
  to_table(df)

In [ ]:
display_listing(sub_by_property, query=[Q_SUB])

,addr,area,bed,car,bath,gas,pool,price,auction,inspect,sub,latitude,longitude
pid,,,,,,,,,,,,,
2016332804,52 Elizabeth Street,417.0,4,2,3,N,N,Auction,07-18T10:30,06-27T09:3,ARTARMON,-33.804157,151.182083
2016261030,23 Stewart Street,353.0,3,2,2,Y,N,Auction,07-04T14:15,06-27T13:3,ARTARMON,-33.803280,151.186874


In [ ]:
display_listing(sub_by_price, query=[Q_GAS,Q_NO_POOL])

,addr,area,bed,car,bath,gas,pool,price,auction,inspect,sub,latitude,longitude
pid,,,,,,,,,,,,,
2016320206,5 Mitchell Crescent,948.0,5,2,3,Y,N,Auction Saturday 18 July 12.00pm,07-18T12:00,06-24T11:0,WARRAWEE,-33.737423,151.115200
2016318837,74 Bannockburn Road,1137.0,6,2,4,Y,N,"Auction Saturday 11 July, 11.30am",06-11T11:30,06-24T11:0,PYMBLE,-33.733334,151.139100
2016321641,18 Oak Street,746.0,5,2,3,Y,N,"Auction Saturday 18 July, 12pm",07-18T12:00,06-25T11:4,NORMANHURST,-33.729380,151.088684
2016270387,115 Edgeworth David Avenue,702.0,4,2,3,Y,N,Contact Agent,None,None,WAHROONGA,-33.707590,151.114761
2016170122,House 7/402 Mona Vale Road,NaN,5,2,2,Y,N,"For Sale $1,800,000",None,06-24T11:3,ST IVES,-33.711063,151.174576
2016308636,2 Staddon Close,923.0,3,2,2,Y,N,Auction 4th July,07-04T14:30,06-24T14:0,ST IVES,-33.716600,151.167145
2016261196,2 Glenelg Place,NaN,4,2,3,Y,N,Auction Saturday 4 July,None,06-24T13:3,ST IVES,-33.708800,151.160400
2014358390,114 Burns Road,1385.0,4,2,3,Y,N,For Sale | Price Guide contact agent,None,None,WAHROONGA,-33.715540,151.135468
2016228780,44 Rushall Street,NaN,5,2,3,Y,N,"Price Guide $2,350,000 - $2,500,000",None,06-24T11:3,PYMBLE,-33.733227,151.140274


### Single Property

In [ ]:
prop_id = 2016281273

In [ ]:
prop = PropertyReport(client,prop_id)

In [ ]:
prop.check_price(price_range=PriceRange(min=1.5e6, max=4e6), price_step=1000)

11:48:44,766 DomainApi INFO start min_price=1500000.0 max_price=4000000.0
11:48:44,767 DomainApi INFO min_price=1500000.0 max_price=4000000.0 mid_price=2750000.0
11:48:49,837 DomainApi INFO min_price=2750000.0 max_price=4000000.0 mid_price=3375000.0
11:48:52,159 DomainApi INFO min_price=2750000.0 max_price=3375000.0 mid_price=3062500.0
11:48:54,494 DomainApi INFO min_price=2750000.0 max_price=3062500.0 mid_price=2906250.0
11:48:56,802 DomainApi INFO min_price=2750000.0 max_price=2906250.0 mid_price=2828125.0
11:48:59,156 DomainApi INFO min_price=2750000.0 max_price=2828125.0 mid_price=2789062.0
11:49:04,502 DomainApi INFO min_price=2789062.0 max_price=2828125.0 mid_price=2808593.0
11:49:06,851 DomainApi INFO min_price=2789062.0 max_price=2808593.0 mid_price=2798827.0
11:49:11,542 DomainApi INFO min_price=2798827.0 max_price=2808593.0 mid_price=2803710.0
11:49:13,899 DomainApi INFO min_price=2798827.0 max_price=2803710.0 mid_price=2801268.0
11:49:16,244 DomainApi INFO min_price=2798827.

(2799437.0, 1500000.0, 4000000.0)

In [ ]:
#pd.set_option('display.max_colwidth', 150)
prop.show_report()

,
address,"23 Stewart Street, Artarmon NSW 2064"
estimate price,"2,549,804.0"
bed-bath-car,3-2-2
list_time,2020-05-14
summary,Beautifully modern family home in community address


## Coding Area

In [1]:
import json
import requests # this library is awesome: http://docs.python-requests.org/en/master/
import re, string, timeit
import time
from collections import namedtuple
import pandas as pd
import numpy as np
import math
import sys, logging
from IPython.display import display, HTML

In [ ]:
class Suburb(object):
  def __init__(self, name, postcode, state='NSW'):
    self.name = name
    self.postcode = postcode
    self.state = state

class PropertyRange(object):
  # bed/bath/car: "3-4" or "3-" (min) or "-4"
  def __init__(self, bed=None, bath=None, car=None):
    self.bed = bed
    self.bath = bath
    self.car = car

PriceRange = namedtuple('PriceRange', 'min max')

def parse_range_str(input):
  if input[-1]=="+":
    min = int(input[:-1])
    max = None
  elif input[-1]=="-":
    min = None
    max = int(input[:-1])
  else:
    result = input.split('-')
    min = int(result[0])
    max = int(result[1]) if len(result) >  1 else min
  return min, max

def update_price_criteria(criteria_dict, price_range):
  if price_range.min:
    criteria_dict.update({"minPrice": int(price_range.min)})
  if price_range.max:
    criteria_dict.update({"maxPrice": int(price_range.max)})
  return criteria_dict

def update_property_criteria(criteria_dict, property_range):
  if property_range.bed:
    min_bed, max_bed = parse_range_str(property_range.bed)
    if min_bed : criteria_dict.update({"minBedrooms": min_bed})
    if max_bed : criteria_dict.update({"maxBedrooms": max_bed})
  if property_range.bath:
    min_bath, max_bath = parse_range_str(property_range.bath)
    if min_bath : criteria_dict.update({"minBathrooms": min_bath})
    if max_bath : criteria_dict.update({"maxBathrooms": max_bath})
  if property_range.car:
    min_car, max_car = parse_range_str(property_range.car)
    if min_car : criteria_dict.update({"minCarspaces": min_car})
    if max_car : criteria_dict.update({"maxCarspaces": max_car})
  return criteria_dict 

In [ ]:
def log(level, msg, *args, **kwargs):
  my_logger().log(level, msg, *args, **kwargs)

def setLogLevel(level):
  my_logger().setLevel(level)  

def init_logging():
  logging.basicConfig(
      format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
      datefmt='%H:%M:%S',
      level=logging.INFO)

def my_logger():
  return logging.getLogger('DomainApi') 

In [ ]:
# domain account
class SiteInfo:
  #client_id ='YOUR_CLIENT_ID'
  #client_secret = 'YOUR_CLIENT_SECRET'
  token_url = 'https://auth.domain.com.au/v1/connect/token'
  api_root_url = 'https://api.domain.com.au'
  ep_listing_residential = '/v1/listings/residential/_search'
  ep_prop_by_id = '/v1/listings/{id}'
  ep_price_estimate = '/v1/properties/{id}/priceEstimate'

In [ ]:
class DomainApiClient(object):
  def __init__(self, site_info):
    self._info = site_info
    self._token = self.get_token()
    self._auth = {"Authorization":"Bearer "+ self._token}
  
  def get_token(self):
    r = requests.post(self._info.token_url, 
                             data = {'client_id':self._info.client_id,
                                     "client_secret":self._info.client_secret,
                                     "grant_type":"client_credentials",
                                     "scope":"api_listings_read",
                                     "Content-Type":"text/json"})
    token = r.json()
    return token['access_token']

  def _get(self, url):
    r = requests.get(url,headers=self._auth)
    return r.json()

  def get(self, end_point):
    url = ''.join([self._info.api_root_url, end_point])
    return self._get(url)

  def _post(self, url, data):
    r = requests.post(url, headers=self._auth, json=data)
    return r.json()

  def post(self, end_point, data):
    url = ''.join([self._info.api_root_url, end_point])
    return self._post(url, data)

  def paged_post(self, end_point, data, on_paged_data=None):
    max_page_size = 100
    page_num = 1
    total_items = 0
    processed_items = 0
    waiting_interval = 2
    url = ''.join([self._info.api_root_url, end_point])
    while page_num > 0:
      data.update({"pageNumber": page_num, "pageSize": max_page_size})
      r = requests.post(url, headers=self._auth, json=data)

      if 'X-Total-Count' in r.headers and total_items < int(r.headers['X-Total-Count']):
        total_items = int(r.headers['X-Total-Count']) # update total items

      if on_paged_data:
        on_paged_data(r.json())
      
      processed_items += len(r.json())      
      page_num = page_num + 1 if processed_items < total_items else -1

      log(logging.DEBUG, 'waiting %s seconds for next page: total=%s processed=%s page_num=%s', waiting_interval, total_items, processed_items, page_num)
      time.sleep(waiting_interval)

    return total_items, processed_items

In [ ]:
class DomainClient(DomainApiClient):
  def __init__(self, site_info):
    super().__init__(site_info)

  def on_new_list_page(self, data):
    log(logging.DEBUG, 'get new page:%s', len(data))
    log(logging.DEBUG, 'new page data:%s', data)

    result = [extract_property_info(x) for x in data]
    log(logging.DEBUG, 'get extracted data:%s', len(result))
    log(logging.DEBUG, 'new extracted data:%s', result)
    log(logging.DEBUG, 'before cached data:%s', len(self._cached_listing))
    if self._cached_listing:
      self._cached_listing += result
    else:
      self._cached_listing = result
    log(logging.DEBUG, 'after cached data:%s', len(self._cached_listing))

  def property_detail(self,prop_id):
    end_point = self._info.ep_prop_by_id.format(id=prop_id)
    result = self.get(end_point)
    ad = result['addressParts']
    geo = result['geoLocation']    

    return {'addr'    : ad['displayAddress'],
            'sub'     : ad['suburb'],
            'street'  : ad['street'],
            'state'   : ad['stateAbbreviation'],
            'postcode': ad['postcode'],
            'bath'    : int(result['bathrooms']),
            'bed'     : int(result['bedrooms']),
            'car'     : int(result['carspaces']),
            'list_time' : result['dateListed'][0:10],
            'desc'    : result['description'],
            'summary' : result['headline'],
            'geo'     : [float(geo['latitude']), float(geo['longitude'])],
            'url'     : result['seoUrl'],
            'type'    : result['propertyTypes']
    }

  def prop_price_estimate(self, prop_id):
    end_point = self._info.ep_price_estimate.format(id=prop_id)
    result = self.get(end_point)
    log(logging.DEBUG, 'get price_estimate %s', result)
    try: low = result['lowerPrice']
    except: low = None
    try: high = result['upperPrice']
    except: high = None
    try: mid = result['midPrice']
    except: mid = None
    return PriceRange(min=low, max=high)

  def listing(self, suburb, property_type = ["House"] ,price_range=None, property_range=None, expand_search=True):
    data = {"listingType":"Sale",
            "propertyTypes":property_type,
    }
    if price_range:
      update_price_criteria(data, price_range)
    if property_range:
      update_property_criteria(data, property_range)
    
    data.update(            
        {"locations":[
              {
                "state":suburb.state,
                "region":"",
                "area":"",
                "suburb":suburb.name,
                "postCode":suburb.postcode,
                "includeSurroundingSuburbs":expand_search
              }
        ]}
    )
    return self.listing_by_criteria(data)

  def listing_by_criteria(self, data):
    log(logging.DEBUG, 'listing criteria:%s', data)
    self._cached_listing = []  # clear list cache
    end_point = self._info.ep_listing_residential
    total, retrieved = self.paged_post(end_point, data, on_paged_data=self.on_new_list_page)
    log(logging.DEBUG, 'total=%s, retrieved=%s, extracted=%s', total, retrieved, len(self._cached_listing))
    return self._cached_listing


In [ ]:
def extract_property_info(data):  
  prop = data['listing']
  pid = prop['id']
  #print(pid)
  pd = prop['propertyDetails']
  try: bed = int(pd['bedrooms'])
  except: bed = None
  try: car = int(pd['carspaces'])
  except: car = None
  try: bath = int(pd['bathrooms'])
  except: bath = None
  addr = pd['displayableAddress']
  simple_addr = ''.join(addr.split(',')[:-1])
  sub = pd['suburb']
  url = 'https://www.domain.com.au/' + prop['listingSlug'] #https://www.domain.com.au/29-somerset-avenue-turramurra-nsw-2074-2016308815  
  clickable_addr = '<a target="_blank" href="{}">{}</a>'.format(url, simple_addr)
  latitude = pd['latitude']
  longitude = pd['longitude']
  #postcode= pd ['postcode']
  gas = 'Y' if 'Gas' in pd['features'] else 'N'
  pool = 'Y' if 'SwimmingPool' in pd['features'] else 'N'
  try: area = int(math.floor(pd['landArea']))
  except: area = None
  try: auction = prop['auctionSchedule']['time'][5:16]
  except: auction = None
  try: inspection = prop['inspectionSchedule']['times'][0]['openingTime'][5:15] 
  except: inspection=None  
  try: price = prop['priceDetails']['displayPrice'] 
  except: price=None
  return [pid, simple_addr, area, bed, car, bath, gas, pool, price, auction, inspection, url, sub, latitude, longitude]
  

In [ ]:
def process_properties(listing_result):
  result = [extract_property_info(x) for x in listing_result]
  return result

def make_clickable(val):
  # target _blank to open new window  
  return '<a target="_blank" href="{}">{}</a>'.format(val[0], val[1])

def create_hyperlink(url, val):
  # target _blank to open new window  
  return '<a target="_blank" href="{}">{}</a>'.format(url, val)

def hyperlink_addr(addr):
  if isinstance(addr, str) and '@' in addr:
    val, url = addr.split('@')
    return '<a target="_blank" href="{}">{}</a>'.format(url, val)
  else:
    return addr

def to_dataframe(prop_list):
  labels = ['pid', 'addr','area', 'bed', 'car', 'bath', 'gas', 'pool', 'price', 'auction', 'inspect', 'url', 'sub', 'latitude','longitude']
  df = pd.DataFrame.from_records(prop_list, columns=labels)
  df.set_index('pid', inplace=True)
  return df

def to_table(df):  
  _df = df.copy()
  #_df['addr'] = _df[['addr','url']].agg('@'.join, axis=1)
  _df['addr'] = _df[['url','addr']].agg(make_clickable, axis=1)
  _df.drop(columns=['url'], inplace=True)  
  #_df = _df.style.format({'addr': hyperlink_addr, 'area':"{:.0f}", 'bed':"{:.0f}",'car':"{:.0f}", 'bath':"{:.0f}"})
  html = HTML(_df.to_html(escape=False)) 
  display(html)

In [ ]:
class PropertyReport(object):
  def __init__(self, client, id):
    self.prop_id = id
    self.client = client
    self.prop_info = self.client.property_detail(self.prop_id)
    self.estimate_price = None

  def is_in_price_range(self, min_price, max_price):
    criteria = {
      "listingType":"Sale",
      "minPrice":int(min_price),
      "maxPrice":int(max_price),
      "propertyTypes":self.prop_info['type'],
      "minBedrooms":self.prop_info['bed'],
      "maxBedrooms":self.prop_info['bed'],
      "minBathrooms":self.prop_info['bath'],
      "maxBathrooms":self.prop_info['bath'],
      "minCarspaces":self.prop_info['car'],
      "maxCarspaces":self.prop_info['car'],
      "locations":[
        {
          "state":self.prop_info['state'],
          "region":"",
          "area":"",
          "suburb":self.prop_info['sub'],
          "postCode":self.prop_info['postcode'],
          "includeSurroundingSuburbs":False
        }
      ]
    }
    result = self.client.listing_by_criteria(criteria)
    return any(self.prop_id==x[0] for x in result)

  def check_price(self, price_range=None, price_step=5000):
    if price_range is None:
      price_range = self.client.prop_price_estimate(self.prop_id)
 
    min_price = price_range.min if price_range.min else 0
    max_price = price_range.max if price_range.max else 0
    mid_price = None
    log(logging.INFO, 'start min_price=%s max_price=%s', min_price, max_price)

    while(min_price and max_price and min_price + price_step < max_price):
      mid_price = min_price + int((max_price - min_price)/2)
      log(logging.INFO, 'min_price=%s max_price=%s mid_price=%s', min_price, max_price, mid_price)
      if self.is_in_price_range(min_price, mid_price):
        max_price = mid_price
      elif self.is_in_price_range(mid_price, max_price): 
        min_price = mid_price
      else:
        log(logging.INFO, 'stop min_price=%s max_price=%s mid_price=%s', min_price, max_price, mid_price)
        break

    self.estimate_price = mid_price
    return mid_price, price_range.min, price_range.max

  def show_report(self):
    report = {
        'address' : create_hyperlink(self.prop_info['url'],self.prop_info['addr']),
        'estimate price' : '{:,}'.format(self.estimate_price) if self.estimate_price else None,
        'bed-bath-car' : '{bed}-{bath}-{car}'.format(bed=self.prop_info['bed'],bath=self.prop_info['bath'],car=self.prop_info['car']),
        'list_time': self.prop_info['list_time'],
        'summary' : self.prop_info['summary'],
        #'decription':'<br>'.join(self.prop_info['desc'].split('.')),
    }
    col_name=''
    df = pd.DataFrame.from_dict(data=report, orient='index', columns=[col_name])
    html = HTML(df.to_html(escape=False, justify='left'))    
    display(html)  

## DEUBG Area

In [ ]:
a={'bed':1, 'show':'dfa<br>dfadgfdgagdfa', 'next':'{num:,.0f}'.format(num=2222), 'text':'<a href="{0}">link</a>'.format('https://colab.research.google.com/')}
df=pd.DataFrame.from_dict(data=a, orient='index', columns=[''])

In [ ]:
display(HTML(df.to_html(escape=False)))


,
bed,1
show,dfadfadgfdgagdfa
next,"2,222"
text,link


In [ ]:
prop_id = '2016290780'
prop_info = client.property_detail(prop_id)

AttributeError: ignored

In [ ]:
a=2.5e6
a

2500000.0

In [ ]:
prop_list = process_properties(sub_list)

TypeError: ignored

In [ ]:
df_prop = to_table(prop_list)

In [ ]:
df_prop

,addr,area,bed,car,bath,gas,pool,price,auction,inspect,latitude,longitude
pid,,,,,,,,,,,,
2016308815,29 Somerset Avenue,936,5,2,2,N,Y,"Auction Saturday 4 July, 12pm",07-04T12:00,06-17T11:0,-33.712933,151.152908
2016308837,80 The Chase Road,929,3,1,1,N,Y,"Auction Saturday 4 July, 2pm",07-04T14:00,06-17T15:0,-33.720310,151.139435
2016308959,5 Stratton Place,nan,3,2,2,Y,Y,For Sale Contact Agent,None,06-17T13:0,-33.691250,151.149628
2016307572,38 Maxwell Street,956,3,2,2,Y,N,Auction Saturday 4 July at 2.15pm,None,06-13T13:3,-33.749874,151.114300
2016284956,29 Murdoch Street,nan,4,2,2,N,N,Sold Off Market,None,None,-33.725185,151.145538
2016247434,53 Kissing Point Road,918,4,1,2,N,N,"AUCTION | Price Guide $1,850,000",06-20T11:00,06-13T13:1,-33.738216,151.123718
2016077007,42 Ashburton Avenue,1220,4,2,2,N,N,Contact Agent,None,None,-33.761100,151.108300
2016296004,37 Duff Street,667,4,1,2,Y,N,Auction Saturday 27 June at 3pm,06-27T15:00,06-13T11:0,-33.734947,151.123459
2016283665,12 Billabong Avenue,1069,3,1,1,N,N,"Auction Saturday 27 June, 1pm",06-27T13:00,06-17T12:3,-33.724770,151.136520


In [ ]:
get_prop = client.get('/v1/listings/2016308959')

In [ ]:
get_prop

{'addressParts': {'displayAddress': '5 Stratton Place, Turramurra NSW 2074',
  'displayType': 'fullAddress',
  'postcode': '2074',
  'stateAbbreviation': 'nsw',
  'street': 'Stratton Place',
  'streetNumber': '5',
  'suburb': 'Turramurra'},
 'advertiserIdentifiers': {'advertiserId': 29538,
  'advertiserType': 'agency',
  'agentIds': ['A87073', 'A108775'],
  'contactIds': [1379098, 1589843]},
 'bathrooms': 2.0,
 'bedrooms': 3.0,
 'carspaces': 2.0,
 'channel': 'residential',
 'dateListed': '2020-06-11T08:43:27Z',
 'dateUpdated': '2020-06-11T08:43:27.523Z',
 'description': "A rare offering in tightly held 'Eden Ridge', a serene village enclave reserved exclusively for over-55s, this beautifully presented residence combines the qualities of a house with the ease and convenience of villa living. Set amid park-like gardens, residents can enjoy the communal recreation areas - swimming pool, clubhouse, and BBQ facilities  whilst being only moments to transport, golf and the beautiful waterways

In [ ]:
sub_list[2]

{'listing': {'advertiser': {'bannerUrl': 'https://images.domain.com.au/img/Agencys/29538/banner_29538.jpeg',
   'contacts': [{'name': 'Andy Howden',
     'photoUrl': 'https://images.domain.com.au/img/29538/contact_1379098.jpeg?mod=200611-184253'},
    {'name': 'Alexander Liapis',
     'photoUrl': 'https://images.domain.com.au/img/29538/contact_1589843.jpeg?mod=200611-184253'}],
   'id': 29538,
   'logoUrl': 'https://images.domain.com.au/img/Agencys/29538/logo_29538.jpeg',
   'name': 'Stone Real Estate Turramurra',
   'preferredColourHex': '#293038',
   'type': 'Agency'},
  'hasFloorplan': True,
  'hasVideo': False,
  'headline': 'Spacious Torrens title villa in over-55s estate',
  'id': 2016308959,
  'inspectionSchedule': {'byAppointment': False,
   'recurring': False,
   'times': [{'closingTime': '2020-06-13T13:40:00',
     'openingTime': '2020-06-13T13:00:00'},
    {'closingTime': '2020-06-17T13:40:00',
     'openingTime': '2020-06-17T13:00:00'},
    {'closingTime': '2020-06-20T13:40